#### Функции предобработки данных (для ручного запуска и тестирования)

In [2]:
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv, find_dotenv

In [ ]:
load_dotenv()

In [ ]:
dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

In [67]:
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

In [68]:
# Извелечение исходных данных

def extract(conn):
    sql = f"""
        select
            f.id, f.floor, f.kitchen_area, f.living_area, f.rooms, f.is_apartment, f.studio, f.total_area, f.price,
            b.build_year, b.building_type_int, b.latitude, b.longitude, b.ceiling_height, b.flats_count, b.floors_total, b.has_elevator
        from flats as f
        left join buildings as b on f.building_id = b.id
        """

    data = pd.read_sql(sql, conn)
    return data

In [69]:
# Предварительная трансформация объединенной таблицы

def transform(data):
    # Изменяем тип булевских столбцов на integer
    bool_cols = data.select_dtypes(bool).columns
    data[bool_cols] = data[bool_cols].astype(int)
    
    # Переименовываем колонку id у квартир на flat_id (т.к. id - это индексная колонка в БД)
    data.rename(columns={'id': 'flat_id'}, inplace=True)
    
    # Удаляем строки с пустыми, отрицательными и нулевыми ценами
    data = data[~(data['price'].isnull() | (data['price'] <= 0))]
    
    # Дабавляем target = ln(1 + price)
    data['target'] = np.log1p(data['price'])
    
    return data

In [70]:
# Удаление строк-дубликатов

def remove_duplicates(data):
    cols_to_check = data.columns.drop(['flat_id']).tolist()
    duplicated_rows = data.duplicated(subset=cols_to_check, keep=False)
    data = data[~duplicated_rows].reset_index(drop=True)
    return data

In [71]:
# Заполнение пропусков в признаках

def fill_missing_values(data):
    cols_with_nans = data.isnull().sum()
    cols_with_nans = cols_with_nans[cols_with_nans > 0].index
    
    # В колонке flat_id не может быть пропусков, т.к. она была индексом в исходной таблице flats
    # Также сюда не попадают колонки price и target, т.к. ранее мы удалили строки с пустыми ценами
    for col in cols_with_nans:
        if data[col].dtype in ['float']:
            fill_value = data[col].mean()
        elif data[col].dtype in ['int', 'object']:
            fill_value = data[col].mode().iloc[0]
        
        data[col].fillna(value=fill_value, inplace=True)
    
    return data

In [72]:
# Удаление строк с выбросами у вещественных признаков

def remove_outliers(data):
    num_cols = data.select_dtypes(['float']).drop(columns=['target']).columns
    threshold = 1.5
    potential_outliers = pd.DataFrame()
    
    for col in num_cols:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        margin = threshold * IQR
        lower = Q1 - margin
        upper = Q3 + margin
        potential_outliers[col] = ~data[col].between(lower, upper)
        
    outliers = potential_outliers.any(axis=1)
    return data[~outliers]

Объединяем исходные данные в одну таблицу

In [17]:
data_1 = extract(dst_conn)
dst_conn.dispose()

In [18]:
data_1.head()

,id,floor,kitchen_area,living_area,rooms,is_apartment,studio,total_area,price,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
0,0,9,9.9,19.900000,1,False,False,35.099998,9500000,1965,6,55.717113,37.781120,2.64,84,12,True
1,1,7,0.0,16.600000,1,False,False,43.000000,13500000,2001,2,55.794849,37.608013,3.00,97,10,True
2,2,9,9.0,32.000000,2,False,False,56.000000,13500000,2000,4,55.740040,37.761742,2.70,80,10,True
3,3,1,10.1,43.099998,3,False,False,76.000000,20000000,2002,4,55.672016,37.570877,2.64,771,17,True
4,4,3,3.0,14.000000,1,False,False,24.000000,5200000,1971,1,55.808807,37.707306,2.60,208,9,True


In [24]:
data_1.shape

(141362, 18)

Меняем тип булевских столбцов на integer, переименовываем id квартир на flat_id и добавляем новый target

In [19]:
data_2 = transform(data_1)

In [31]:
data_2.head()

,flat_id,floor,kitchen_area,living_area,rooms,is_apartment,studio,total_area,price,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator,target
0,0,9,9.9,19.900000,1,0,0,35.099998,9500000,1965,6,55.717113,37.781120,2.64,84,12,1,16.066802
1,1,7,0.0,16.600000,1,0,0,43.000000,13500000,2001,2,55.794849,37.608013,3.00,97,10,1,16.418200
2,2,9,9.0,32.000000,2,0,0,56.000000,13500000,2000,4,55.740040,37.761742,2.70,80,10,1,16.418200
3,3,1,10.1,43.099998,3,0,0,76.000000,20000000,2002,4,55.672016,37.570877,2.64,771,17,1,16.811243
4,4,3,3.0,14.000000,1,0,0,24.000000,5200000,1971,1,55.808807,37.707306,2.60,208,9,1,15.464169


In [43]:
data_2.shape

(141362, 18)

Проверяем наличие дубликатов

In [50]:
cols_to_check = data_2.columns.drop(['flat_id']).tolist()
duplicated_rows = data_2.duplicated(subset=cols_to_check, keep=False)
duplicated_data = data_2[duplicated_rows]
print('Кол-во строк-дубликатов: ', len(duplicated_data))

Кол-во строк-дубликатов:  17425


Удаляем дубликаты

In [52]:
data_3 = remove_duplicates(data_2)

In [59]:
data_3.shape

(123937, 18)

Проверяем наличие пропусков данных

In [60]:
data_3.isnull().sum()

flat_id              0
floor                0
kitchen_area         0
living_area          0
rooms                0
is_apartment         0
studio               0
total_area           0
price                0
build_year           0
building_type_int    0
latitude             0
longitude            0
ceiling_height       0
flats_count          0
floors_total         0
has_elevator         0
target               0
dtype: int64

In [62]:
data_4 = fill_missing_values(data_3)

In [63]:
data_4.shape

(123937, 18)

Удаляем строки с выбросами у вещественных признаков

In [65]:
data_5 = remove_outliers(data_4)

In [66]:
data_5.shape

(106341, 18)